## Import required libraries

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.globals import set_verbose, set_debug

## Set the required variables and file imports

In [2]:
load_dotenv()

True

In [3]:
# Disable verbose logging
set_verbose(False)

# Disable debug logging
set_debug(False)

In [4]:
# OPENAI API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Setup the model and Langchain template

In [5]:
model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o-mini", temperature=0.7)

In [6]:
template = """
## TASK
Answer the question based only on the context provided below

## CONTEXT
{context}

## QUESTION 
{question}
"""

# Langchain prompt template
prompt = ChatPromptTemplate.from_template(template)

## Setting up the Embedding Model

In [7]:
query = "Some question to be asked"

In [8]:
embeddings = OpenAIEmbeddings()

## PineCone setup

In [ ]:
index_name = "<put the index name which was given>"
namespace = "<put the namespace name which was given>"

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings, namespace=namespace)

In [ ]:
threshold = 0.80

for i in range(len(queries)):
    print("User:", queries[i])
    response = vectorstore.similarity_search_with_score(queries[i], k=10)

    context = ""

    for j in range(len(response)):
        if response[j][1]>threshold:
            context += response[j][0].page_content + "\n\n"
    
    if context=="":
        print("Sorry, I cannot help you with that question.")
        print("\n")
        print(f'----------------------------------End of Question {i+1}------------------------------------')
        print("\n")
        continue
    
    prompt_formatted_str: str = prompt.format(context=context, question=queries)
    
    answer = model.predict(prompt_formatted_str)
    print("Agent Reply:", answer)
    print("\n")
    print("--------------------------------------Context---------------------------------------------")
    for k in range(len(response)):
        print(response[k])
    print(f'----------------------------------End of Question {i+1}------------------------------------')
    print("\n")